In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json

import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd
#import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import shutil
import keras
from PIL import Image
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import re
import string

tfds.disable_progress_bar()
from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D,Reshape, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, UpSampling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Input
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
AUTO = tf.data.AUTOTUNE
from sklearn.metrics import confusion_matrix
import random

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 128
EPOCHS = 50
CROP_TO = 32
SEED = 26

PROJECT_DIM = 128
LATENT_DIM = 512
WEIGHT_DECAY = 0.0005

In [ ]:
!pip install tensorflow-addons

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
#train_labels = to_categorical(train_labels)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255
test_images = test_images / 255 

x_train, x_unlab, y_train, y_unlab = train_test_split( train_images, train_labels , test_size=0.5, random_state=42 )
X_train, X_test, Y_train, Y_test = train_test_split( x_train,y_train , test_size=0.2, random_state=40 )

In [17]:
len(x_train)

50000

In [4]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.02),
        layers.experimental.preprocessing.RandomWidth(0.2),
        layers.experimental.preprocessing.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(X_train)

In [5]:


num_classes = 10
input_shape = (32, 32, 3)

def create_encoder():
    resnet = tf.keras.applications.ResNet50V2( include_top=False, weights=None, input_shape=input_shape, pooling="avg" )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model

encoder = create_encoder()
encoder.summary()


Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [6]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [7]:
learning_rate = 0.0001
batch_size = 512
hidden_units = 512
projection_units = 256
num_epochs = 2
dropout_rate = 0.5
temperature = 0.05

In [8]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):

        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)

        logits = tf.divide( tf.matmul(  
            feature_vectors_normalized, tf.transpose(feature_vectors_normalized)),self.temperature,)
        
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    
    return model

In [11]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=X_train, y=Y_train, batch_size=batch_size, epochs=2)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functional  (None, 2048)             23564807  
 )                                                               
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
Total params: 24,089,351
Trainable params: 24,043,904
Non-trainable params: 45,447
_________________________________________________________________
Epoch 1/2


KeyboardInterrupt: 

In [116]:
classifier = create_classifier(encoder, trainable=False) 

history = classifier.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=num_epochs) 

accuracy = classifier.evaluate(X_test, Y_test)[1] 
print(f"Test accuracy: {round(accuracy * 100, 2)}%") 

Epoch 1/2
157/157 [==============================] - 13s 47ms/step - loss: nan - sparse_categorical_accuracy: 0.1015
Epoch 2/2
157/157 [==============================] - 4s 21ms/step - loss: nan - sparse_categorical_accuracy: 0.1044
Test accuracy: 10.44%


In [14]:
# Helper function for merging new history objects with older ones
def append_history(losses, val_losses, accuracy, val_accuracy, history):
    losses = losses + history.history["loss"]
    val_losses = val_losses + history.history["val_loss"]
    accuracy = accuracy + history.history["accuracy"]
    val_accuracy = val_accuracy + history.history["val_accuracy"]
    return losses, val_losses, accuracy, val_accuracy


# Plotter function
def plot_history(losses, val_losses, accuracies, val_accuracies):
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(accuracies)
    plt.plot(val_accuracies)
    plt.legend(["train_accuracy", "val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [ ]:


def resmodel():
    
    
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3),))
    model.add(layers.Conv2D(64, (3, 3),))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3),))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(.2))
    model.add(layers.Flatten()) 
    model.add(layers.Dense(32))

    model.summary()
    return model

In [12]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
AUTO = tf.data.AUTOTUNE

def create_model(X,Y,X_test, Y_test,num_epochs):
    
    data_augmentation.layers[0].adapt(X)
    
    encoder = create_encoder()

    encoder_with_projection_head = add_projection_head(encoder)
    
    encoder_with_projection_head.compile(optimizer=keras.optimizers.Adam(learning_rate),
                                     loss=SupervisedContrastiveLoss(temperature),)
    
    
    history = encoder_with_projection_head.fit(x=X, y=Y, batch_size=256, epochs=num_epochs)
    
    
    classifier = create_classifier(encoder, trainable=False) 

    history = classifier.fit(x=X, y=Y, batch_size=batch_size, epochs=num_epochs) 

    accuracy = classifier.evaluate(X_test, Y_test)[1] 
    
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    #model.summary()
    return encoder,classifier

In [13]:
arr = np.concatenate((X_train, x_unlab))
X_all = arr
arr = np.concatenate((Y_train, y_unlab))
Y_all = arr

In [14]:
# callbacks=[keras.callbacks.EarlyStopping(patience=4, verbose=1), ],

def train_full_model(X_train, X_test, Y_train, Y_test,num):
    
    X_train, Y_train = shuffle(X_train, Y_train)
    
    encoder,classifier = create_model(X_train,Y_train,X_test, Y_test,num)

    accuracy = classifier.evaluate(X_test, Y_test)[1] 
    print(f"Test accuracy: {round(accuracy * 100, 2)}%") 
    
    
    return encoder,classifier


encoder,classifier = train_full_model(X_all, X_test, Y_all, Y_test,1)

KeyboardInterrupt: 

In [15]:
encoder,classifier = train_full_model(X_train, X_test, Y_train, Y_test,5)

Epoch 1/5
79/79 [==============================] - 715s 9s/step - loss: 5.5968
Epoch 2/5
79/79 [==============================] - 708s 9s/step - loss: 5.4473
Epoch 3/5
79/79 [==============================] - 637s 8s/step - loss: 5.3939
Epoch 4/5
79/79 [==============================] - 677s 9s/step - loss: 5.3373
Epoch 5/5
79/79 [==============================] - 697s 9s/step - loss: 5.2956
Epoch 1/5
40/40 [==============================] - 43s 899ms/step - loss: 2.3356 - sparse_categorical_accuracy: 0.1674
Epoch 2/5
40/40 [==============================] - 36s 899ms/step - loss: 2.0685 - sparse_categorical_accuracy: 0.2226
Epoch 3/5
40/40 [==============================] - 39s 979ms/step - loss: 1.9947 - sparse_categorical_accuracy: 0.2400
Epoch 4/5
40/40 [==============================] - 38s 936ms/step - loss: 1.9620 - sparse_categorical_accuracy: 0.2474
Epoch 5/5
157/157 [==============================] - 12s 71ms/step - loss: 1.7884 - sparse_categorical_accuracy: 0.3212
Test accu

In [ ]:


def train_active_learning_models(
    encoder,
    classifier,
    X_train,
    Y_train,
    x_unlab,
    y_unlab,
    X_test,
    Y_test,
    num_iterations=1,
    num_epochs = 1
):

    # Creating lists for storing metrics
    losses, val_losses, accuracies, val_accuracies = [], [], [], []

    data_augmentation.layers[0].adapt(X_train)
    
    history = encoder_with_projection_head.fit(x=X_train, y=Y_train, batch_size=256, epochs=num_epochs)
    
    classifier = create_classifier(encoder, trainable=False) 

    history = classifier.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=num_epochs) 

    accuracy = classifier.evaluate(X_test, Y_test)[1]
    
    d = 100/num_iterations
    l = len(y_unlab)
    x = int(np.round( l/d ))
    
    for iteration in range(num_iterations-1):
        
        l = len(y_unlab)
        
        print("\n\n")
        
        print("Iteration : ")
        
        print(iteration)
        
        print("\n\n")
         
        #generate random number and substract from all numbers
        rnd = random.sample(range(1, l), x)
        all = list(range(1, l))
        main_list = list(set(all) - set(rnd))
        
        #add those index to from unlablled set to training set
        new_lab = x_unlab[rnd]
        arr = np.concatenate((X_train, new_lab))
        X_train = arr

        #predict on the set and add to training data
        new_y = np.round(model.predict(new_lab))
        arr = np.concatenate((Y_train, new_y))
        Y_train = arr
        
        
        #create the new unlabelled set
        x_unlab = x_unlab[main_list]

        #create the new unlabelled label set
        y_unlab = y_unlab[main_list]
        
        #test on data
        test(model, X_test, Y_test)
        
        #train on data
        model = train(model,X_train, Y_train)
        
    #Final round training set
    arr = np.concatenate((X_train, x_unlab))
    X_train = arr

    #Final round training label set
    new_y = np.round(model.predict(x_unlab))
    arr = np.concatenate((Y_train, new_y))
    Y_train = arr
    
    #train final
    model = train(model,X_train, Y_train)
    
    #test for final time
    test(model, X_test, Y_test)
        
    #return model
    return model

active_learning_model = train_active_learning_models(encoder,classifier,X_train,Y_train,x_unlab,y_unlab,X_test,Y_test,num_iterations=5)

In [ ]:
def test(model, X_test, Y_test):
    print("-" * 100)
    print("Test set evaluation: ", model.evaluate( X_test, Y_test , verbose=0, return_dict=True), )
    print("-" * 100)